In [1]:
class PartridgeTilingSolver:
    """
    Solves the Partridge Tiling problem.
    A Partridge Tiling of order n involves tiling a square grid of side S = n(n+1)/2
    with a specific set of smaller square tiles:
    - 1 tile of size 1x1
    - 2 tiles of size 2x2
    - ...
    - n tiles of size n x n

    The program finds all completely filled tiling grids that satisfy an optionally
    provided partially filled grid.
    """

    def __init__(self, n, partially_filled_grid_input=None):
        """
        Initializes the solver.
        Args:
            n (int): The Partridge number.
            partially_filled_grid_input (list of lists of int, optional):
                Represents the S x S grid. 0 means empty/unconstrained.
                A positive integer k (1 to n) means the cell must be covered by a kxk tile.
        """
        if not isinstance(n, int) or n <= 0:
            raise ValueError("n must be a positive integer.")
        self.n = n
        self.S = n * (n + 1) // 2
        self.solutions = []

        self.grid = [[0 for _ in range(self.S)] for _ in range(self.S)]

        if partially_filled_grid_input:
            if not isinstance(partially_filled_grid_input, list) or \
                    len(partially_filled_grid_input) != self.S or \
                    not all(isinstance(row, list) and len(row) == self.S for row in partially_filled_grid_input):
                raise ValueError(f"Partially filled grid must be a list of lists representing an {self.S}x{self.S} grid.")

            for r_idx in range(self.S):
                for c_idx in range(self.S):
                    val = partially_filled_grid_input[r_idx][c_idx]
                    if not isinstance(val, int) or val < 0 or val > self.n:
                        raise ValueError(
                            f"Invalid value {val} in partially_filled_grid at ({r_idx},{c_idx}). "
                            f"Must be 0 or an integer between 1 and {self.n}."
                        )
            self.partial_grid = partially_filled_grid_input
        else:
            self.partial_grid = [[0 for _ in range(self.S)] for _ in range(self.S)]

        # tile_counts_needed[k] stores how many k x k tiles are left to place.
        # Index 0 is unused for convenience with 1-based k (tile sizes).
        self.tile_counts_needed = [0] * (self.n + 1)
        for i in range(1, self.n + 1):
            self.tile_counts_needed[i] = i

    def _can_place(self, r_start, c_start, size):
        """Checks if a tile of 'size x size' can be placed at (r_start, c_start)."""
        if r_start + size > self.S or c_start + size > self.S:
            return False

        for r in range(r_start, r_start + size):
            for c in range(c_start, c_start + size):
                if self.grid[r][c] != 0:  # Overlaps with an already placed tile
                    return False
                if self.partial_grid[r][c] != 0 and self.partial_grid[r][c] != size:
                    return False # Inconsistent with partial_grid constraint for this cell
        return True

    def _place_tile(self, r_start, c_start, size):
        """Places a tile of 'size x size' at (r_start, c_start) on self.grid."""
        for r in range(r_start, r_start + size):
            for c in range(c_start, c_start + size):
                self.grid[r][c] = size
        self.tile_counts_needed[size] -= 1

    def _remove_tile(self, r_start, c_start, size):
        """Removes a tile of 'size x size' from (r_start, c_start) on self.grid."""
        for r in range(r_start, r_start + size):
            for c in range(c_start, c_start + size):
                self.grid[r][c] = 0
        self.tile_counts_needed[size] += 1

    def _find_next_empty_cell(self):
        """Finds the next empty cell (r, c) in self.grid."""
        for r_idx in range(self.S):
            for c_idx in range(self.S):
                if self.grid[r_idx][c_idx] == 0:
                    return r_idx, c_idx
        return -1, -1 # No empty cells

    def _backtrack(self):
        """Recursive backtracking function."""
        r, c = self._find_next_empty_cell()

        if r == -1: # Base case: Grid is full
            if all(count == 0 for count in self.tile_counts_needed[1:]): # Check all tiles used correctly
                self.solutions.append([row[:] for row in self.grid])
            return

        # If the current empty cell (r,c) has a specific constraint from partial_grid
        if self.partial_grid[r][c] != 0:
            size_to_try = self.partial_grid[r][c]
            if self.tile_counts_needed[size_to_try] > 0 and self._can_place(r, c, size_to_try):
                self._place_tile(r, c, size_to_try)
                self._backtrack()
                self._remove_tile(r, c, size_to_try)
        else: # No constraint from partial_grid for cell (r,c)
            # Try placing available tiles of different sizes (largest first is a heuristic)
            for size_to_try in range(self.n, 0, -1):
                if self.tile_counts_needed[size_to_try] > 0:
                    if self._can_place(r, c, size_to_try):
                        self._place_tile(r, c, size_to_try)
                        self._backtrack()
                        self._remove_tile(r, c, size_to_try)

    def find_solutions(self):
        """
        Starts the search for all Partridge Tiling patterns.
        Returns:
            list of list of lists of int: A list of all found solutions.
                                         Each solution is an S x S grid.
        """
        self.solutions = [] # Reset for multiple calls
        self._backtrack()
        return self.solutions

# --- Example Usage ---
if __name__ == '__main__':

    print("\n--- Solving for n=9 with partial fill ---")
    # partial_n1_filled = [
    #                      [0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,4,4,4,4,8,8,8,8,8,8,8,8],
    #                      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,5,5,5,5,5,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,5,5,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,5,5,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,5,5,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,5,5,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7],
    #                      [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7]
    #                      ]
    partial_n1_filled = [
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,6,6,6,6,6,6,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,8,8],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,4,4,4,4,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,4,4,4,4,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,4,4,4,4,0,0,0,0,2,2,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,4,4,4,4,0,0,0,0,2,2,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,5,5,5,5,5,5,5,5,5,5,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,5,5,5,5,5,5,5,5,5,5,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,5,5,5,5,5,5,5,5,5,5,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,5,5,5,5,5,5,5,5,5,5,9,9,9,9,9,9,9,9,9],
                        [9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,5,5,5,5,5,5,5,5,5,5,9,9,9,9,9,9,9,9,9]
    ]
    solver_n1_partial = PartridgeTilingSolver(n=9, partially_filled_grid_input=partial_n1_filled)
    solutions_n1_partial = solver_n1_partial.find_solutions()
    if solutions_n1_partial:
        for i, sol in enumerate(solutions_n1_partial):
            print(f"Solution {i+1} for n=9 (partial):")
            for row in sol:
                print(row)
            print("-" * 10)
    else:
        print("No solutions found for n=9 with this partial grid.")



--- Solving for n=9 with partial fill ---
No solutions found for n=9 with this partial grid.
